In [1]:
from statsmodels.tsa.stattools import grangercausalitytests
import csv
import requests
import pandas as pd
import numpy as np
import os

In [2]:
gtrends_data = pd.read_csv("gtrends_data/gtrends_weekly_01-01-12_10-29-18.csv", names=["ds", "y"])
gtrends_data['y'] = gtrends_data['y'].replace({"<1": "0"})

In [3]:
POLONIEX_OHLCV_BASEURL = 'https://poloniex.com/public?command=returnChartData&currencyPair='

"""
https://poloniex.com/public?command=returnChartData&currencyPair=BTC_POT&start=1435699200&end=9999999999&period=14400

"""

def get_ohlcv_poloniex(pair='USDT_ETH', start=1435699200, end=9999999999, period=900):
    """
    returns ohlcv data for poloniex as pandas dataframe
    convert to unix timestamp using https://coderstoolbox.net/unixtimestamp/
    :param pair: str pair on poloniex
    :param start: int unix timestamp of beginning time
    :param end: int unix timestamp of ending time
    :param period: int candle width in seconds
    :return: pandas df of ohlcv data from poloniex for specified pair, times, and period
    """
    query = POLONIEX_OHLCV_BASEURL + pair + '&start=' + str(start) + '&end=' + str(end) + '&period=' + str(period)
    resp = requests.get(query)

    if resp.status_code != 200:
        raise requests.ApiError('GET /tasks/ {}'.format(resp.status_code))

    return pd.DataFrame(resp.json())

In [4]:
df = get_ohlcv_poloniex(pair='USDT_BTC', start=0, end=9999999999, period=900)
df = df[['date','close','volume']]
df['ds'] = pd.to_datetime(df['date'], unit='s')
df['log'] = np.log(df['close'])
df['vol'] = df['log'].rolling(5).std()
df.dropna();

In [5]:
path = 'gtrends_data/'

for series in ['close','volume','vol']:
    for i in  [1,2,4,6,8]:
        for isAbs in [True, False]:
            data = df[series]
            if isAbs:
                data.apply(lambda x:abs(x))
            combined = pd.concat([data,gtrends_data['y'][2:]], axis=1)
            combined = combined.dropna(how='any')
            print (series)
            print (i)
            print (grangercausalitytests(combined, i))

close
1

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.0630  , p=0.3057  , df_denom=78, df_num=1
ssr based chi2 test:   chi2=1.1039  , p=0.2934  , df=1
likelihood ratio test: chi2=1.0964  , p=0.2951  , df=1
parameter F test:         F=1.0630  , p=0.3057  , df_denom=78, df_num=1
{1: ({'ssr_ftest': (1.062981224047147, 0.3057225876793295, 78.0, 1), 'ssr_chi2test': (1.1038651172797296, 0.2934194346848047, 1), 'lrtest': (1.0964110462010694, 0.2950550809211057, 1), 'params_ftest': (1.0629812240470633, 0.3057225876793507, 78.0, 1.0)}, [<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x1c167c5710>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x1c167c5780>, array([[0., 1., 0.]])])}
close
1

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.0630  , p=0.3057  , df_denom=78, df_num=1
ssr based chi2 test:   chi2=1.1039  , p=0.2934  , df=1
likelihood ratio test: chi2=1.0964  , p=0.2951  , df=1
p

close
6

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.0630  , p=0.3057  , df_denom=78, df_num=1
ssr based chi2 test:   chi2=1.1039  , p=0.2934  , df=1
likelihood ratio test: chi2=1.0964  , p=0.2951  , df=1
parameter F test:         F=1.0630  , p=0.3057  , df_denom=78, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.2208  , p=0.3008  , df_denom=75, df_num=2
ssr based chi2 test:   chi2=2.6045  , p=0.2719  , df=2
likelihood ratio test: chi2=2.5630  , p=0.2776  , df=2
parameter F test:         F=1.2208  , p=0.3008  , df_denom=75, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=2.7931  , p=0.0464  , df_denom=72, df_num=3
ssr based chi2 test:   chi2=9.1939  , p=0.0268  , df=3
likelihood ratio test: chi2=8.6971  , p=0.0336  , df=3
parameter F test:         F=2.7931  , p=0.0464  , df_denom=72, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.2254  , p=0.9233

        0.]])])}
close
8

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.0630  , p=0.3057  , df_denom=78, df_num=1
ssr based chi2 test:   chi2=1.1039  , p=0.2934  , df=1
likelihood ratio test: chi2=1.0964  , p=0.2951  , df=1
parameter F test:         F=1.0630  , p=0.3057  , df_denom=78, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.2208  , p=0.3008  , df_denom=75, df_num=2
ssr based chi2 test:   chi2=2.6045  , p=0.2719  , df=2
likelihood ratio test: chi2=2.5630  , p=0.2776  , df=2
parameter F test:         F=1.2208  , p=0.3008  , df_denom=75, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=2.7931  , p=0.0464  , df_denom=72, df_num=3
ssr based chi2 test:   chi2=9.1939  , p=0.0268  , df=3
likelihood ratio test: chi2=8.6971  , p=0.0336  , df=3
parameter F test:         F=2.7931  , p=0.0464  , df_denom=72, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0

volume
2

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.8654  , p=0.3551  , df_denom=78, df_num=1
ssr based chi2 test:   chi2=0.8987  , p=0.3431  , df=1
likelihood ratio test: chi2=0.8937  , p=0.3445  , df=1
parameter F test:         F=0.8654  , p=0.3551  , df_denom=78, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.4046  , p=0.6687  , df_denom=75, df_num=2
ssr based chi2 test:   chi2=0.8632  , p=0.6495  , df=2
likelihood ratio test: chi2=0.8585  , p=0.6510  , df=2
parameter F test:         F=0.4046  , p=0.6687  , df_denom=75, df_num=2
{1: ({'ssr_ftest': (0.8654078107045098, 0.3551000272160141, 78.0, 1), 'ssr_chi2test': (0.8986927265008371, 0.3431324811972904, 1), 'lrtest': (0.8937438129383395, 0.34446478260020874, 1), 'params_ftest': (0.8654078107045222, 0.3551000272160091, 78.0, 1.0)}, [<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x1c1e63f908>, <statsmodels.regression.linear_model.Regressio

parameter F test:         F=0.1622  , p=0.9755  , df_denom=66, df_num=5

Granger Causality
number of lags (no zero) 6
ssr based F test:         F=0.1607  , p=0.9861  , df_denom=63, df_num=6
ssr based chi2 test:   chi2=1.1633  , p=0.9787  , df=6
likelihood ratio test: chi2=1.1545  , p=0.9791  , df=6
parameter F test:         F=0.1607  , p=0.9861  , df_denom=63, df_num=6
{1: ({'ssr_ftest': (0.8654078107045098, 0.3551000272160141, 78.0, 1), 'ssr_chi2test': (0.8986927265008371, 0.3431324811972904, 1), 'lrtest': (0.8937438129383395, 0.34446478260020874, 1), 'params_ftest': (0.8654078107045222, 0.3551000272160091, 78.0, 1.0)}, [<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x1c1e66b940>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x1c1e669240>, array([[0., 1., 0.]])]), 2: ({'ssr_ftest': (0.404608654308501, 0.6686854868875393, 75.0, 2), 'ssr_chi2test': (0.8631651291914688, 0.6494804362347474, 2), 'lrtest': (0.8585417677458054, 0.650983564

parameter F test:         F=0.1622  , p=0.9755  , df_denom=66, df_num=5

Granger Causality
number of lags (no zero) 6
ssr based F test:         F=0.1607  , p=0.9861  , df_denom=63, df_num=6
ssr based chi2 test:   chi2=1.1633  , p=0.9787  , df=6
likelihood ratio test: chi2=1.1545  , p=0.9791  , df=6
parameter F test:         F=0.1607  , p=0.9861  , df_denom=63, df_num=6

Granger Causality
number of lags (no zero) 7
ssr based F test:         F=0.0767  , p=0.9992  , df_denom=60, df_num=7
ssr based chi2 test:   chi2=0.6707  , p=0.9986  , df=7
likelihood ratio test: chi2=0.6677  , p=0.9986  , df=7
parameter F test:         F=0.0767  , p=0.9992  , df_denom=60, df_num=7

Granger Causality
number of lags (no zero) 8
ssr based F test:         F=0.0742  , p=0.9997  , df_denom=57, df_num=8
ssr based chi2 test:   chi2=0.7711  , p=0.9993  , df=8
likelihood ratio test: chi2=0.7671  , p=0.9993  , df=8
parameter F test:         F=0.0742  , p=0.9997  , df_denom=57, df_num=8
{1: ({'ssr_ftest': (0.865407

       [0., 0., 0., 0., 0., 0., 0., 1., 0.]])])}
vol
6

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.3030  , p=0.5836  , df_denom=76, df_num=1
ssr based chi2 test:   chi2=0.3150  , p=0.5746  , df=1
likelihood ratio test: chi2=0.3144  , p=0.5750  , df=1
parameter F test:         F=0.3030  , p=0.5836  , df_denom=76, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.1516  , p=0.8596  , df_denom=73, df_num=2
ssr based chi2 test:   chi2=0.3241  , p=0.8504  , df=2
likelihood ratio test: chi2=0.3234  , p=0.8507  , df=2
parameter F test:         F=0.1516  , p=0.8596  , df_denom=73, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.2846  , p=0.8363  , df_denom=70, df_num=3
ssr based chi2 test:   chi2=0.9393  , p=0.8159  , df=3
likelihood ratio test: chi2=0.9336  , p=0.8173  , df=3
parameter F test:         F=0.2846  , p=0.8363  , df_denom=70, df_num=3

Granger Causality
number of lags (no zero) 4

parameter F test:         F=0.1532  , p=0.9783  , df_denom=64, df_num=5

Granger Causality
number of lags (no zero) 6
ssr based F test:         F=0.1527  , p=0.9878  , df_denom=61, df_num=6
ssr based chi2 test:   chi2=1.1118  , p=0.9810  , df=6
likelihood ratio test: chi2=1.1035  , p=0.9814  , df=6
parameter F test:         F=0.1527  , p=0.9878  , df_denom=61, df_num=6

Granger Causality
number of lags (no zero) 7
ssr based F test:         F=0.1999  , p=0.9843  , df_denom=58, df_num=7
ssr based chi2 test:   chi2=1.7612  , p=0.9719  , df=7
likelihood ratio test: chi2=1.7403  , p=0.9728  , df=7
parameter F test:         F=0.1999  , p=0.9843  , df_denom=58, df_num=7

Granger Causality
number of lags (no zero) 8
ssr based F test:         F=0.1641  , p=0.9947  , df_denom=55, df_num=8
ssr based chi2 test:   chi2=1.7186  , p=0.9885  , df=8
likelihood ratio test: chi2=1.6984  , p=0.9889  , df=8
parameter F test:         F=0.1641  , p=0.9947  , df_denom=55, df_num=8
{1: ({'ssr_ftest': (0.303023